1. First start the ipcluster using the notebook $\color{blue}{\text{start_ipcluster.ipynb}}$. Change the number of cores if required.
2. Import the ipyparallel Client

In [1]:
from ipyparallel import Client
rc=Client(profile='mpi')

1. Start each cell with the magic command $\color{red}{\%\%px}$, else the cell won't run in parallel!!
2. Load standard modules like 'pyfftw', 'numpy', etc.
3. Load modules for current computations from the folder: $\color{red}{\text{'ClassRepository'}}$ 
4. Start the MPI communicator

In [2]:
%%px
#2.
from mpi4py import MPI
import sys
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
#3.
sys.path.insert(0,'ClassRepository')
from DataDownload import DataDownload
#4.
comm = MPI.COMM_WORLD
my_id = comm.Get_rank()
nproc = comm.Get_size()

if(my_id==0):
    sys.stdout.write('Starting with processors:%d\n' %(int(nproc)))

[stdout:0] Starting with processors:8


1. Get domain size from the JHTDB database
2. Divide the data along X-axis across the processors

In [3]:
%%px
#1.
nx=isotropic1024coarse['nx']
ny=isotropic1024coarse['ny']
nz=isotropic1024coarse['nz']
#2.
nx=nx//nproc
sys.stdout.write('MPI id={0:4d} nx={1:6d} ny={2:6d} nz={3:6d}\n'. \
    format(my_id,nx,ny,nz))

[stdout:0] MPI id=   0 nx=   128 ny=  1024 nz=  1024
[stdout:1] MPI id=   2 nx=   128 ny=  1024 nz=  1024
[stdout:2] MPI id=   5 nx=   128 ny=  1024 nz=  1024
[stdout:3] MPI id=   6 nx=   128 ny=  1024 nz=  1024
[stdout:4] MPI id=   4 nx=   128 ny=  1024 nz=  1024
[stdout:5] MPI id=   3 nx=   128 ny=  1024 nz=  1024
[stdout:6] MPI id=   1 nx=   128 ny=  1024 nz=  1024
[stdout:7] MPI id=   7 nx=   128 ny=  1024 nz=  1024


1. Set the authorization token for downloading the data from JHTDB
2. Set the name and time instant of the dataset to be downloaded
3. Create an instance of the downloader and then use it to call the download function

In [4]:
%%px
#1.
auth_token='edu.jhu.meneveau-hiSmxkae'
#2.
dataset_name='isotropic1024coarse'
time=0.0
#3.
myDDL=DataDownload()
comm.Barrier(); t1=MPI.Wtime()
vx,vy,vz=myDDL.DownldData4Velocity(dataset_name,time,nx,ny,nz,nproc,my_id,auth_token)
comm.Barrier(); t2=MPI.Wtime()
if(my_id==0):
    sys.stdout.write('Downloading cost: %0.2f seconds; with processors:%d\n' %(t2-t1,int(nproc)))

CompositeError: one or more exceptions from call to method: execute
[Engine Exception]EngineError: Engine 0 died while running task u'faad2186-2ab4831be525e977fb036405'
[Engine Exception]EngineError: Engine 1 died while running task u'ac828586-cf4fa016efe501b48ee1efea'
[Engine Exception]EngineError: Engine 2 died while running task u'b60964bc-fcd3d08bf8a195280e44608b'
[Engine Exception]EngineError: Engine 3 died while running task u'48b5d5b1-b42d55141ca69e5d322d1839'
.... 4 more exceptions ...

1. To store the downloaded data, first $\color{red}{\text{create a folder manually in the scratch folder}}$, and set its path against the dirName
2. Provide file name initial. The file name will get appended by the core id
3. Call the function $\color{blue}{\text{SaveDataOnVM}}$ to store the data

In [ ]:
%%px
#1.
dirName='/home/admin/dataSnapshot/'
#2.
fileNameInitial='veldata_t0p0_myID'
#3.
comm.Barrier(); t1=MPI.Wtime()
myDDL=DataDownload()
myDDL.SaveVelDataOnVM(dirName,fileNameInitial,vx,vy,vz,nproc,my_id)
comm.Barrier(); t2=MPI.Wtime()
if(my_id==0):
    sys.stdout.write('Writing cost: %0.2f seconds; with processors:%d\n' %(t2-t1,int(nproc)))